## 2024 -25 Roster Explore Notebook #1
#### Created 8/25/24 -by JBS

Scraping code taken from Roster_scraper_variable.ipynb in the original college_hockey folder

In [12]:

url = 'https://www.collegehockeynews.com/reports/roster/Michigan-State/32'

import os
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re


# print list of folders one dir up from current
# print(os.listdir(os.path.join(os.getcwd(), '..')))

# path ro TEMP folder
temp_folder = os.path.join(os.getcwd(), '..', 'TEMP')
# Data folder
data_folder = os.path.join(os.getcwd(), '..', 'data')
# Image folder
img_folder = os.path.join(os.getcwd(), '..', 'images')

# # save file into each folder for testing

# file = os.path.join(temp_folder, 'test.txt')
# with open(file, 'w') as f:
#     f.write('test')

### Get All Scheduled games for the given season in CHN Data

In [13]:
# Set the URL to scrape
url_base = 'https://www.collegehockeynews.com/schedules/?season=' # Link to the 2022-2023 season with all results

#### THESE NEED TO BE UPDATED EACH SEASON ####
#### Set the season varriable portion of the url 
url_season = '20242025'
#### string to add to file names and whatnote
file_tag = '2024_current'

############# Set the output folder to the TEMP folder for testing
final_output_dir = os.path.join(temp_folder)

# # Set the output folder to the data folder
# final_output_dir = os.path.join(data_folder, 'schedule')


# temp_output_dir = '../TEMP/results/'



#########################################################################


## Construction the url to feel to parser
url = url_base + url_season

# Get the page with requests
response = requests.get(url)

# Create a BeautifulSoup object
soup = BeautifulSoup(response.text, 'html.parser')

# select the table or tables
tables = soup.find_all('table')

# Initialize variables
current_date = None
current_conference = None
game_notes = None

# Initialize an empty list to hold the data
data = []

# Parse the table with BeautifulSoup

rows = soup.find_all('tr')

# Loop through each row to find relevant information
for row in rows:
    # Check for date row
    if row.get('class') == ['stats-section']:
        current_date = row.find('td').text.strip()
    # Check for conference row
    elif row.get('class') == ['sked-header']:
        current_conference = row.find('td').text.strip()
    # Check for game notes
    elif len(row.find_all('td')) == 2:
        game_notes = row.find_all('td')[1].text.strip()
    # Process rows with game data
    elif row.get('valign') == 'top':
        cells = row.find_all('td')
        if len(cells) >= 9:
            home_team = cells[0].text.strip()
            home_team_link = cells[0].find('a')['href'] if cells[0].find('a') else None
            home_score = cells[1].text.strip()
            away_team = cells[3].text.strip()
            away_team_link = cells[3].find('a')['href'] if cells[3].find('a') else None
            away_score = cells[4].text.strip()
            ot = cells[5].text.strip()
            box_link = cells[7].find('a')['href'] if cells[7].find('a') else None
            metrics_link = cells[8].find('a')['href'] if cells[8].find('a') else None
             # Capture Game Notes
            game_notes_cell = cells[-1].find('small')
            game_notes = game_notes_cell.text.strip() if game_notes_cell else None

            # Append data to the list
            data.append([current_date, current_conference, game_notes, home_team, home_team_link, home_score, away_team, away_team_link, away_score, ot, box_link, metrics_link])
            game_notes = None  # Reset game notes for the next row
            

# Create a DataFrame
columns = ['Date', 'Conference', 'Game_Notes', 'Away_Team', 'Away_Team_Link', 'Away_Score', 'Home_Team', 'Home_Team_Link', 'Home_Score',  'OT', 'Box_Link', 'Metrics_Link']
df = pd.DataFrame(data, columns=columns)




### Some Data Tranformation
#### Creates Unique GAME_ID
- Cleaning string formats, 
- create team abbv

In [14]:
## Extract the day of the week from the date and save in new column
df['Day'] = pd.to_datetime(df['Date']).dt.day_name()
# remove day of the week from date
# format data column as YYYY-MM-DD
df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')

### Create a new column for the game ID
## Game ID will be a combination of the date and abbreviated team names

# Function to abbreviate the team names
for row in df.itertuples():
    home_team = row.Home_Team
    away_team = row.Away_Team
    home_team_abbr = home_team.split(' ')[-1]
    away_team_abbr = away_team.split(' ')[-1]
    game_id = f'{row.Date}_{home_team_abbr}_{away_team_abbr}'
    df.loc[row.Index, 'Game_ID'] = game_id

# Create a new column for the game ID
df['Game_ID'] = df['Game_ID'].str.replace(',', '')

# Apply the function to the DataFrame
df['Game_ID'] = df.apply(lambda row: f'{row.Date}_{row.Home_Team}_{row.Away_Team}', axis=1)

# STORE AS SCHED_DF
sched_df = df.copy()

# Save the DataFrame to a CSV file
# IN THE TEMP FOLDER
sched_df.to_csv(os.path.join(final_output_dir, f'schedule_{file_tag}.csv'), index=False)
# TO THE DATA FOLDER
# sched_df.to_csv(os.path.join(data_folder, f'schedule_{file_tag}.csv'), index=False)



In [15]:
sched_df.head()

,Date,Conference,Game_Notes,Away_Team,Away_Team_Link,Away_Score,Home_Team,Home_Team_Link,Home_Score,OT,Box_Link,Metrics_Link,Day,Game_ID
0,2024-10-04,Non-Conference,,Michigan State,/reports/team/Michigan-State/32,,Lake Superior,/reports/team/Lake-Superior/24,,,None,None,Friday,2024-10-04_Lake Superior_Michigan State
1,2024-10-04,Non-Conference,,Minnesota State,/reports/team/Minnesota-State/35,,Michigan,/reports/team/Michigan/31,,,None,None,Friday,2024-10-04_Michigan_Minnesota State
2,2024-10-04,Non-Conference,,Arizona State,/reports/team/Arizona-State/61,,Air Force,/reports/team/Air-Force/1,,,None,None,Friday,2024-10-04_Air Force_Arizona State
3,2024-10-05,Non-Conference,,Providence,/reports/team/Providence/46,,Union,/reports/team/Union/54,,,None,None,Saturday,2024-10-05_Union_Providence
4,2024-10-05,Non-Conference,,Penn State,/reports/team/Penn-State/60,,Alaska,/reports/team/Alaska/4,,,None,None,Saturday,2024-10-05_Alaska_Penn State


## Get The Roster Data

In [16]:

## Load results for season to get team links
# Read the CSV file
# results = pd.read_csv('../../data/results_table_all_time.csv')
results = sched_df.copy()

season = 2024

# # Concatenate the unique values in 'Home_Team_Link' and 'Away_Team_Link'
team_links = pd.concat([results['Home_Team_Link'], results['Away_Team_Link']])

# Drop duplicates and NaN values
team_links = team_links.drop_duplicates().dropna()

# Clean up the links - drop everything before the third slash - keep the team name and the school ID as string
team = team_links.str.split('/', expand=True)[3]
# ORIGINAL
number = team_links.str.split('/', expand=True)[4]

# NEW - Create a string for the year '{season}{season+1}'
year = str(season) + str(season+1)
# reconstruct the link and store in a new column
team_links = 'https://www.collegehockeynews.com/reports/roster/' + team + '/' + number + '/' + year

# reset the index
team_links = team_links.reset_index(drop=True)






team_links


# Output results
print(f'Number of team links: {len(team_links)}')
print(f'TEST LINK: FIRST IN LIST: {team_links[0]}')
print(f'TEST LINK: LAST IN LIST: {team_links.iloc[-1]}')


Number of team links: 64
TEST LINK: FIRST IN LIST: https://www.collegehockeynews.com/reports/roster/Lake-Superior/24/20242025
TEST LINK: LAST IN LIST: https://www.collegehockeynews.com/reports/roster/Long-Island/62/20242025


## Functions that clean the roster dataframes

In [17]:
df.head()# Function to split "Last Team" into "Team" and "League" with edge case handling
def split_last_team(last_team):
    # Use regular expression to extract team and league
    match = re.search(r'(.+) \((.+)\)', last_team)
    if match:
        return match.groups()
    else:
        # If no league is specified, return the team as is and leave league blank
        return last_team, ""

# Updated function to correctly capture the player's position and handle edge cases in "Last Team"
def parse_and_transform_roster(html_content):
    # Initialize BeautifulSoup object
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Find the table with the roster
    roster_table = soup.find('table', {'id': 'players'})

    # Check if the table exists
    if roster_table is None:
        print("Skipped: The table doesn't exist.")
        return None
    
    # Extract headers
    header_row = roster_table.find('thead').find('tr')
    headers = [header.text.strip() for header in header_row.find_all('th')]
    headers.append('Position')  # Add the Position column to headers
    
    # Initialize data list and current_position variable
    data = []
    current_position = None  # Initialize as None to later filter out irrelevant rows
    
    # Iterate through each row in the table
    for row in roster_table.find_all('tr'):
        if 'class' in row.attrs and 'stats-section' in row.attrs['class']:
            current_position = row.text.strip()
        else:
            cells = row.find_all('td')
            if cells and current_position:
                row_data = [cell.text.strip() for cell in cells]
                row_data.append(current_position)  # Add the current position to the row data
                data.append(row_data)
    
    # Create DataFrame
    df = pd.DataFrame(data, columns=headers)

    # Check if DataFrame is empty or if key columns are missing
    if df.empty or 'Last Team' not in df.columns or 'NHL Draft' not in df.columns:
        print("Skipped: The DataFrame is empty or missing key columns.")
        return None
    
    # Cleanup: Remove rows where 'No.' column is not numeric
    df = df[df['No.'].str.isnumeric()]
    
    # Cleanup: Drop the 'Pos' column
    df.drop(columns=['Pos'], inplace=True)
    
    # Transform Height to Inches
    df['Height_Inches'] = df['Ht.'].apply(convert_to_inches)

    # Transform NHL Draft to Draft_Year, NHL_Team, and D_Round
    # Transform NHL Draft to Draft_Year, NHL_Team, and D_Round
    draft_result = df['NHL Draft'].apply(split_nhl_draft)

    # Check if there are enough values to unpack
    if len(draft_result) > 0:
        df['Draft_Year'], df['NHL_Team'], df['D_Round'] = zip(*draft_result)
    else:
        # Handle the case when result is empty
        df['Draft_Year'], df['NHL_Team'], df['D_Round'] = [None] * len(df), [None] * len(df), [None] * len(df)

    df.drop(columns=['NHL Draft'], inplace=True) # Drop the original NHL Draft column
    
    # Handle edge cases in "Last Team" to split into "Team" and "League"
    df['Team'], df['League'] = zip(*df['Last Team'].apply(split_last_team))
    df.drop(columns=['Last Team'], inplace=True)
    
    # Rename the trouble column Hometown\nLast Team\nNHL Draft
    df.rename(columns={'Hometown\nLast Team\nNHL Draft': 'Hometown'}, inplace=True)

    # assign data types No. Wt. and Height_Inches to int, DOB to datetime
    int_list = ['No.', 'Wt.', 'Height_Inches']
    # Convert columns to numeric, coercing errors to NaN
    df[int_list] = df[int_list].apply(pd.to_numeric, errors='coerce')

    # Replace NaNs with a default value for specific columns
    default_value = 0
    df[int_list] = df[int_list].fillna(value=default_value)

    # Confirm that NaNs are filled
    print(df[int_list].isna().sum())  # Should output all zeros

    # Convert the columns to integers
    df[int_list] = df[int_list].astype(int)

    # df = df[df['Height_Inches'].notna()]  # Assuming convert_to_inches returns None for bad values

    
    return df

# Function to split "NHL Draft" into "Draft_Year", "NHL_Team", and "D_Round"
def split_nhl_draft(nhl_draft):
    try:
        draft_year, nhl_team, d_round = nhl_draft.split('-')
        return draft_year, nhl_team, d_round
    except ValueError:
        # Handle missing or incomplete data
        return None, None, None

# # Test the function
# test_values = ['2022-WSH-7', '', '2021-DET']
# [split_nhl_draft(val) for val in test_values]



# Function to convert height in "ft-in" format to total inches
def convert_to_inches(height_str):
    try:
        feet, inches = map(int, height_str.split('-'))
        return (feet * 12) + inches
    except ValueError:
        return None

### Run the roster scrape and parser for the list of team name

In [18]:
# Correcting the code to handle the 'Team' and 'Last Team' columns properly

## Loop through the team links and parse the roster data 
### Notes: save the roster link to the dataframe and add the team name and school ID

roster_dfs = []  # Assuming this list exists to store each roster DataFrame

# Extract team names from team_links
team_names = pd.Series(team_links).str.split('/', expand=True)[5]


for i, link in enumerate(team_links):
    print(f'Processing team {i+1} of {len(team_links)}')
    
    try:
        # Make GET request to team link
        r = requests.get(link)
        r.raise_for_status()  # This will raise an HTTPError if the HTTP request returned an unsuccessful status code
        html_content = r.text
        
        # Parse and transform the roster data
        roster_df = parse_and_transform_roster(html_content)
    
    except requests.RequestException as e:
        print(f"Error during the request for team {i+1} ({team_names.iloc[i]}): {e}")
        continue
    
    except Exception as e:
        print(f"Error processing the data for team {i+1} ({team_names.iloc[i]}): {e}")
        continue
    
    # Check if the DataFrame exists (i.e., the page had content)
    if roster_df is None:
        print(f"Skipping team {i+1} due to missing or empty data.")
        continue  # Skip this iteration and move to the next one

    try:
        # Reset the index if it's not unique
        roster_df.reset_index(drop=True, inplace=True)
        
        # Add the team name
        current_team = team_names.iloc[i]  # Extract the current team name
        roster_df['Current Team'] = current_team  # Add it to the DataFrame

        # Add a column with the season
        roster_df['Season'] = season
        
        # Add the roster DataFrame to the list
        roster_dfs.append(roster_df)
    
    except Exception as e:
        print(f"Error working with the DataFrame for team {i+1} ({team_names.iloc[i]}): {e}")
        continue

# Assuming the last dataframe processed is the one of interest (can be adjusted later)
# roster_df = roster_dfs[-1]

# Add all the DataFrames in the list to a single DataFrame
roster_df = pd.concat(roster_dfs, ignore_index=True)

Processing team 1 of 64
No.              0
Wt.              0
Height_Inches    0
dtype: int64
Processing team 2 of 64
No.              0
Wt.              0
Height_Inches    0
dtype: int64
Processing team 3 of 64
No.              0
Wt.              0
Height_Inches    0
dtype: int64
Processing team 4 of 64
No.              0
Wt.              0
Height_Inches    0
dtype: int64
Processing team 5 of 64
No.              0
Wt.              0
Height_Inches    0
dtype: int64
Processing team 6 of 64
No.              0
Wt.              0
Height_Inches    0
dtype: int64
Processing team 7 of 64
No.              0
Wt.              0
Height_Inches    0
dtype: int64
Processing team 8 of 64
No.              0
Wt.              0
Height_Inches    0
dtype: int64
Processing team 9 of 64
No.              0
Wt.              0
Height_Inches    0
dtype: int64
Processing team 10 of 64
No.              0
Wt.              0
Height_Inches    0
dtype: int64
Processing team 11 of 64
No.              0
Wt.            

### Further Roster Cleaning Steps


In [19]:


# Further transformations
roster_df.columns = roster_df.columns.str.replace('.', '')  # Remove periods in column names
roster_df['Name'] = roster_df['Name'].str.split(',').str[::-1].str.join(' ')  # Convert 'Last, First' to 'First Last'
roster_df['Player'] = roster_df['Name']  # Store the Player name as 'First Last'
roster_df['Player'] = roster_df['Player'].str.replace(u'\xa0', u' ').str.strip()  # Cleanup player name
roster_df = roster_df.drop(['Name'], axis=1)  # Drop the original 'Name' column

# Renaming 'Team' to 'Last Team' if it exists in the DataFrame
if 'Team' in roster_df.columns:
    roster_df.rename(columns={'Team': 'Last Team'}, inplace=True)

# Checking if 'Last Team' exists in the dataframe before reordering
if 'Last Team' not in roster_df.columns:
    print("'Last Team' column not found in the DataFrame. Please check the data extraction process.")
else:
    # Reorder columns without duplicate 'Team'
    roster_df = roster_df[['Current Team', 'Player', 'No', 'Position', 'Yr', 'Ht', 'Wt', 'DOB', 
                           'Hometown', 'Height_Inches', 'Draft_Year', 'NHL_Team', 'D_Round', 'Last Team', 'League']]

roster_df.head() if 'Last Team' in roster_df.columns else None

,Current Team,Player,No,Position,Yr,Ht,Wt,DOB,Hometown,Height_Inches,Draft_Year,NHL_Team,D_Round,Last Team,League
0,Lake-Superior,Adam Barone,0,Defensemen,Fr,6-1,174,5/6/2004,"Sault Ste. Marie, Ont.",73,None,None,None,,
1,Lake-Superior,Jack Blanchett,16,Defensemen,So,5-11,185,5/12/2003,"Monroe, Mich.",71,None,None,None,Powell,BCHL
2,Lake-Superior,Mike Brown,0,Defensemen,Jr,6-2,209,4/3/2001,"Belmont, Mass.",74,None,None,None,Merrimack,
3,Lake-Superior,Evan Bushy,5,Defensemen,So,6-1,195,3/26/2002,"Mankato, Minn.",73,None,None,None,Trail,BCHL
4,Lake-Superior,Jacob Conrad,4,Defensemen,So,5-11,180,5/18/2002,"Green Bay, Wis.",71,None,None,None,Fairbanks,NAHL


In [20]:
### CHECK RESULTING DATAFRAME
roster_df.head()

roster_df.info()

roster_df.sample(20)

# Save the DataFrame to a CSV file
# In the TEMP folder
roster_df.to_csv(os.path.join(final_output_dir, f'roster_{file_tag}.csv'), index=False)
# In the DATA folder
roster_df.to_csv(os.path.join(data_folder, f'roster_{file_tag}.csv'), index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1908 entries, 0 to 1907
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Current Team   1908 non-null   object
 1   Player         1908 non-null   object
 2   No             1908 non-null   int32 
 3   Position       1908 non-null   object
 4   Yr             1908 non-null   object
 5   Ht             1908 non-null   object
 6   Wt             1908 non-null   int32 
 7   DOB            1908 non-null   object
 8   Hometown       1908 non-null   object
 9   Height_Inches  1908 non-null   int32 
 10  Draft_Year     234 non-null    object
 11  NHL_Team       234 non-null    object
 12  D_Round        234 non-null    object
 13  Last Team      1908 non-null   object
 14  League         1908 non-null   object
dtypes: int32(3), object(12)
memory usage: 201.4+ KB


In [21]:
# Number of players with no Last Team and or League vs total number of players
missing_team = roster_df['Last Team'].isna().sum()
missing_league = roster_df['League'].isna().sum()

print(f'Number of players with missing Last Team: {missing_team}')
print(f'Number of players with missing League: {missing_league}')
print(f'Total number of players: {len(roster_df)}')
print('########################################\n')
print('########################################')
print(f'Distribution of players by position:\n{roster_df["Position"].value_counts()}')
print('########################################\n')



Number of players with missing Last Team: 0
Number of players with missing League: 0
Total number of players: 1908
########################################

########################################
Distribution of players by position:
Position
Forwards       1097
Defensemen      610
Goaltenders     201
Name: count, dtype: int64
########################################



In [22]:
# Player by League
print('########################################')
print(f'Distribution of players by League:\n{roster_df["League"].value_counts()}')

## Distrobution by class year
print('########################################\n')
print(f'Distribution of players by Class Year:\n{roster_df["Yr"].value_counts()}')



########################################
Distribution of players by League:
League
                  477
USHL              428
NAHL              298
BCHL              209
AJHL               70
HEA                58
NCHC               38
ECAC               37
NTDP               35
CCHA               34
AHA                31
NCDC               31
B10                27
OJHL               22
SJHL               14
Independents       13
MJHL               13
CCHL               11
D-I Ind.            8
SM-sarja            6
J20 Nationell       6
SWE                 3
USHS                3
PREP                3
DI Independent      2
NCAA DIII           2
EHL                 2
MHL                 2
J20                 2
NCAA D3             2
USPHL               2
NA3HL               2
USPHL NCDC          1
ACHA-II             1
D-III               1
Sweden              1
USHS-Prep           1
WCHA                1
Midget AAA          1
SHL                 1
DIII                1
Prep           